# 05 — Regulation Q&A Agent (RAG)

Open-ended question answering over the EU AI Act with inline citations. Demonstrates the core RAG pattern: semantic search → context injection → grounded answer generation.

In [ ]:
import sys
sys.path.insert(0, "..")

from src.tools import init_tools, TOOL_SCHEMAS
from src.agent import run_agent
from src.models import RegulationAnswer
from src.embeddings import get_openai_client

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

In [ ]:
QA_SYSTEM_PROMPT = """Du bist ein Experte für die EU-KI-Verordnung (AI Act). Beantworte Fragen zur Verordnung mithilfe der verfügbaren Tools.

Regeln:
- Stütze deine Antworten immer auf den tatsächlichen Verordnungstext.
- Verwende Inline-Zitate wie [Art. 5(1)], [Art. 50], [Erwägungsgrund 47], [Anhang III].
- Wenn die Verordnung die Frage nicht behandelt, sage das klar.
- Nutze zuerst das Suchtool für allgemeine Anfragen, dann get_article/get_recital für bestimmte Vorschriften.

Antworte abschließend mit einem JSON-Objekt:
{
  "answer": "Deine ausführliche Antwort mit Inline-Zitaten [Art. X]",
  "citations": [{"article": "Art. X", "title": "...", "relevance": "..."}],
  "confidence": "high" | "medium" | "low"
}

Antworte auf Deutsch."""

## Frage 1: Was ist die Definition eines KI-Systems?

In [ ]:
r1, m1 = run_agent(
    client=client,
    system_prompt=QA_SYSTEM_PROMPT,
    user_message="Was ist die Definition eines KI-Systems gemäß der EU-KI-Verordnung?",
    tools=TOOL_SCHEMAS,
)

print("=== Tool-Aufrufe ===")
for m in m1:
    if isinstance(m, dict) and m.get('tool_calls'):
        for tc in m['tool_calls']:
            print(f"  {tc['function']['name']}({tc['function']['arguments'][:60]})")

print(f"\n=== Antwort ===\n{r1[:2000]}")

## Frage 2: Welche Strafen gibt es bei Nichteinhaltung?

In [ ]:
r2, m2 = run_agent(
    client=client,
    system_prompt=QA_SYSTEM_PROMPT,
    user_message="Welche Sanktionen und Bußgelder gibt es bei Nichteinhaltung der EU-KI-Verordnung?",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{r2[:2000]}")

## Frage 3: Ab wann gilt die Verordnung?

In [ ]:
r3, m3 = run_agent(
    client=client,
    system_prompt=QA_SYSTEM_PROMPT,
    user_message="Was sind der Zeitplan und die wichtigsten Daten für das Inkrafttreten der EU-KI-Verordnung?",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{r3[:2000]}")

## Frage 4: KI-Modelle mit allgemeinem Verwendungszweck (GPAI)

In [ ]:
r4, m4 = run_agent(
    client=client,
    system_prompt=QA_SYSTEM_PROMPT,
    user_message="Was sagt die EU-KI-Verordnung über KI-Modelle mit allgemeinem Verwendungszweck (GPAI)? Welche Pflichten haben deren Anbieter?",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{r4[:2000]}")